In [6]:
import numpy as np
import pandas as pd
import re
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [7]:
topics_df = pd.read_json('data/themes.json')
topics_df = topics_df.rename(columns={'Theme': 'topic', 'WordCloud': 'words'})
topics_df.head()

,topic,words
0,"Письма, указы, заявления от госорганов","[письмо, указ, поправка, заявление, сообщение,..."
1,Истец,"[истец, после, основании, иск, иск, суд, требо..."
2,Ответчик,"[против, ответчик, иск, действие, поведение, с..."
3,Преступник,"[преступник, уличить, уличать, разыскивать, об..."
4,Потерпевшая сторона,"[потерпевшая, сторона, покушение, нападение, о..."


In [8]:
queries_df = pd.read_csv('data/queries.txt', sep='\t', names=['id', 'queries'])
queries_df.head()

,id,queries
0,00000000-0000-0000-0000-000000000000,"((Text(""речные круизы"")) AND (Metadata(SourceC..."
1,00000000-0000-0000-0000-000000000000,"((Text(""Поезд"")) AND (Metadata(SourceCategory(..."
2,00000000-0000-0000-0000-000000000000,"((((Exact(Text(""Яндекс.Директ"")))) OR ((Text(""..."
3,00000000-0000-0000-0000-000000000000,"((Text(""отели"")) AND ((Metadata(SourceTopic({3..."
4,00000000-0000-0000-0000-000000000000,"((((Text(""российских""))) OR ((Location(Id(4C8B..."


In [9]:
def extract_and_preprocess_text(query_text):
    pattern = r'(OrderedText|Text|OrderedWildcard|Wildcard)\("(.*?)"\)'
    extracted_text = re.findall(pattern, query_text)
    extracted_text = [text.lower() for _, text in extracted_text]
    
    tokenizer = RegexpTokenizer(r'\w+')
    stop_words = set(stopwords.words("russian"))
    russian_pronouns = ['я', 'ты', 'он', 'она', 'оно', 'мы', 'вы', 'они', 'себя', 'свое', 'своего', 'своей', 'своим', 'своих', 'свою', 'себе', 'мой', 'твой', 'его', 'её', 'ее', 'наш', 'ваш', 'их', 'сам', 'сама', 'само', 'сами', 'твои', 'мои', 'его', 'её', 'наши', 'ваши', 'их']
    stop_words.update(russian_pronouns)
    
    preprocessed_text = []
    for text in extracted_text:
        tokens = tokenizer.tokenize(text)
        filtered_words = [word for word in tokens if word not in stop_words]
        preprocessed_text.extend(filtered_words)
    
    return preprocessed_text

queries_df['words'] = queries_df['queries'].apply(lambda x: extract_and_preprocess_text(x))
queries_df.head()

,id,queries,words
0,00000000-0000-0000-0000-000000000000,"((Text(""речные круизы"")) AND (Metadata(SourceC...","[речные, круизы]"
1,00000000-0000-0000-0000-000000000000,"((Text(""Поезд"")) AND (Metadata(SourceCategory(...",[поезд]
2,00000000-0000-0000-0000-000000000000,"((((Exact(Text(""Яндекс.Директ"")))) OR ((Text(""...","[яндекс, директ, telegram, выделите, фрагмент,..."
3,00000000-0000-0000-0000-000000000000,"((Text(""отели"")) AND ((Metadata(SourceTopic({3...",[отели]
4,00000000-0000-0000-0000-000000000000,"((((Text(""российских""))) OR ((Location(Id(4C8B...",[российских]


In [10]:
import spacy
import numpy as np

# Загрузка модели для русского языка
nlp = spacy.load("ru_core_news_md")

def word_to_vector(word):
    token = nlp(word)
    if token.has_vector:
        vector = token.vector
        return vector
    else:
        return np.zeros(nlp.vocab.vectors_length)  # Возвращаем нулевой вектор, если слово не найдено

def vectorize_words(words):
    vectors = [word_to_vector(word) for word in words]
    return vectors

# Добавление нового столбца "wordvector" в датафреймы
queries_df['vectors'] = queries_df['words'].apply(vectorize_words)
topics_df['vectors'] = topics_df['words'].apply(vectorize_words)

# Вывод первых нескольких строк датафреймов
print("Queries DataFrame:")
display(queries_df.head())

print("Topics DataFrame:")
display(topics_df.head())


Queries DataFrame:


,id,queries,words,vectors
0,00000000-0000-0000-0000-000000000000,"((Text(""речные круизы"")) AND (Metadata(SourceC...","[речные, круизы]","[[-0.33333185, 0.33925715, 0.621041, 0.3348376..."
1,00000000-0000-0000-0000-000000000000,"((Text(""Поезд"")) AND (Metadata(SourceCategory(...",[поезд],"[[0.5008677, -0.6154453, 0.14322977, -0.216310..."
2,00000000-0000-0000-0000-000000000000,"((((Exact(Text(""Яндекс.Директ"")))) OR ((Text(""...","[яндекс, директ, telegram, выделите, фрагмент,...","[[-0.20394225, -0.5428032, 0.17268088, -0.2441..."
3,00000000-0000-0000-0000-000000000000,"((Text(""отели"")) AND ((Metadata(SourceTopic({3...",[отели],"[[0.36390617, -0.5678413, -0.24011786, 0.07641..."
4,00000000-0000-0000-0000-000000000000,"((((Text(""российских""))) OR ((Location(Id(4C8B...",[российских],"[[0.05624003, -0.4158787, -0.039217323, -0.135..."


Topics DataFrame:


,topic,words,vectors
0,"Письма, указы, заявления от госорганов","[письмо, указ, поправка, заявление, сообщение,...","[[-0.4985207, -0.041002605, -0.1494906, 0.2914..."
1,Истец,"[истец, после, основании, иск, иск, суд, требо...","[[-0.75692004, 0.44495207, 0.17751521, 0.15788..."
2,Ответчик,"[против, ответчик, иск, действие, поведение, с...","[[0.18638138, -0.27080196, -0.3037158, 0.13587..."
3,Преступник,"[преступник, уличить, уличать, разыскивать, об...","[[-0.67205477, 0.15145117, -0.029043723, -0.07..."
4,Потерпевшая сторона,"[потерпевшая, сторона, покушение, нападение, о...","[[0.5008677, -0.6154453, 0.14322977, -0.559029..."


In [11]:
import tensorflow as tf
from tensorflow.keras.metrics import CosineSimilarity

In [13]:
def compare_vector_fields(field1, field2):
    normalized_field1 = tf.linalg.l2_normalize(tf.cast(field1, dtype=tf.float32), axis=1)
    normalized_field2 = tf.linalg.l2_normalize(tf.cast(field2, dtype=tf.float32), axis=1)
    dot_product = tf.matmul(normalized_field1, normalized_field2, transpose_b=True)
    magnitude_product = tf.norm(normalized_field1, axis=1)[:, tf.newaxis] * tf.norm(normalized_field2, axis=1)
    epsilon = 1e-8  # Small epsilon value to avoid division by zero
    cosine_similarity = dot_product / (magnitude_product + epsilon)
    average_similarity = tf.reduce_mean(cosine_similarity)

    return average_similarity

def calculate_similarity(query_vector, topics_df):
    similarity_dict = {}

    # Compare the query vector with each topic vector
    for index, row in topics_df.iterrows():
        topic_vector = row['vectors']
        similarity = compare_vector_fields(query_vector, topic_vector)
        topic = row['topic']
        similarity_dict[topic] = similarity

    return similarity_dict

# Call the function with the first query vector and topics_df
query_vector = queries_df['vectors'].iloc[0]
similarity_dict = calculate_similarity(query_vector, topics_df)

# Sort the similarity dictionary by values in descending order
sorted_similarity = sorted(similarity_dict.items(), key=lambda x: x[1], reverse=True)

print(queries_df['words'].iloc[0])
# Print the top 20 values
for topic, similarity in sorted_similarity[:20]:
    print(f"Topic: {topic}, Similarity: {similarity}")


['речные', 'круизы']
Topic: Депозитарий, Similarity: 0.13023903965950012
Topic: Разведение свиней, Similarity: 0.12477616220712662
Topic: Животноводство, Similarity: 0.11677528917789459
Topic: Разведение птицы, Similarity: 0.11090096831321716
Topic: Электротранспорт, Similarity: 0.1086474135518074
Topic: Строительство автодорог, Similarity: 0.10658008605241776
Topic: Экспорт пшеницы, Similarity: 0.10436434298753738
Topic: Тарифы ЖКХ, Similarity: 0.09997227042913437
Topic: Крушение поезда, Similarity: 0.09891384094953537
Topic: Ремонт транспортного оборудования и дорожного покрытия, Similarity: 0.09761718660593033
Topic: Листинг, Similarity: 0.09598755091428757
Topic: Страхование имущества, Similarity: 0.09460227936506271
Topic: Фондовый Рынок, Similarity: 0.09159249067306519
Topic: Отмена или задержка рейса, Similarity: 0.09059879928827286
Topic: Строительство мостов, Similarity: 0.08713724464178085
Topic: Растениеводство, Similarity: 0.08454261720180511
Topic: Электронные технологии в

In [14]:
display(topics_df.loc[topics_df["topic"] == "Депозитарий", "words"])

122    [депозитарий, депозитарий]
Name: words, dtype: object

In [15]:
def calculate_average_vector(vectors):
    average_vector = np.mean(vectors, axis=0)
    return average_vector

queries_df['vector'] = queries_df['vectors'].apply(calculate_average_vector)

def calculate_exponential_average(vectors):
    weights = np.exp(-0.5 * np.arange(len(vectors)))
    weights /= np.sum(weights)
    weighted_vectors = np.multiply(vectors, weights[:, np.newaxis])
    average_vector = np.sum(weighted_vectors, axis=0)
    return average_vector

topics_df['vector'] = topics_df['vectors'].apply(calculate_exponential_average)

c:\Users\haska\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [20]:
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    magnitude_product = np.linalg.norm(vector1) * np.linalg.norm(vector2)
    similarity = dot_product / magnitude_product
    return similarity

# Функция для сравнения вектора запроса с векторами тем
def compare_query_with_topics(query_vector, topic_vectors, topic_names):
    similarity_dict = {}
    
    for topic_vector, topic_name in zip(topic_vectors, topic_names):
        similarity = cosine_similarity(query_vector, topic_vector)
        similarity_dict[topic_name] = similarity
    
    return similarity_dict

# Пример использования функции
query_vector = queries_df['vector'][500]  # Пример вектора запроса
topic_vectors = topics_df['vector']  # Векторы тем
topic_names = topics_df['topic']  # Названия тем

result_dict = compare_query_with_topics(query_vector, topic_vectors, topic_names)
# Сортировка словаря по значениям в порядке убывания
sorted_results = sorted(result_dict.items(), key=lambda x: x[1], reverse=True)

print(queries_df['words'][500])
# Вывод первых 20 элементов
for topic, similarity in sorted_results[:20]:
    print(f"Topic: {topic}, Similarity: {similarity}")

['рпл', 'российская', 'премьер', 'лига']
Topic: Правительство и парламент, Similarity: 0.5403700172913039
Topic: Президент РФ, Similarity: 0.4425656977850679
Topic: Диверсия, Similarity: 0.4120568871542171
Topic: Потерпевшая сторона, Similarity: 0.38921844615276807
Topic: Выборы, Similarity: 0.3782185709024172
Topic: Госрегулирование телекоммуникационной отрасли, Similarity: 0.3736124517492009
Topic: Военные преступления, Similarity: 0.37013784596062155
Topic: Фитосанитарный контроль, Similarity: 0.365494764191881
Topic: Нефтегазовая отрасль, Similarity: 0.34982430711932144
Topic: Мобильная связь, Similarity: 0.32563296760941807
Topic: Отставка, Similarity: 0.32207429318405856
Topic: Господдержка, Similarity: 0.3122921313186043
Topic: Инцидент с посольством, консульством, Similarity: 0.31223564933255893
Topic: Раскрытие государственной или коммерческой тайны, Similarity: 0.3066287261947866
Topic: Судебная экспертиза, Similarity: 0.30549753613910485
Topic: Увольнения и сокращения, Simil

C:\Users\haska\AppData\Local\Temp\ipykernel_1996\1178905693.py:4: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / magnitude_product
